In [1]:
import sys
!{sys.executable} -m pip install optuna

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import sys
sys.path.append('../utils/')
sys.path.append('../paviaUTools/')
# sys.path.insert(1, '../utils')
# sys.path.insert(2, '../paviaUTools')

import matplotlib.pyplot as plt
from datasetLoader import datasetLoader
import os
import numpy as np
from whole_pipeline import whole_pipeline_all, whole_pipeline_divided, whole_pipeline_divided_parallel
import torch
from plots import *
from weights_anal import *
from MetaLearner import HDDOnBands
import consts

import gc

torch.cuda.empty_cache()
gc.collect()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

parent_dir = os.path.join(os.getcwd(),"..")
csv_path = os.path.join(parent_dir, 'datasets', 'paviaU.csv')
gt_path = os.path.join(parent_dir, 'datasets', 'paviaU_gt.csv')
# csv_path = os.path.join(parent_dir, 'datasets', 'pavia.csv')
# gt_path = os.path.join(parent_dir, 'datasets', 'pavia_gt.csv')

dsl = datasetLoader(csv_path, gt_path)

df = dsl.read_dataset(gt=False)
X = np.array(df)
X = X.reshape((610,340, 103))
# X = X.reshape((1096, 715, 102))

df = dsl.read_dataset(gt=True)
y = np.array(df)

X = torch.from_numpy(X)
y = torch.from_numpy(y)

X = X.to(device)
y = y.to(device)

In [3]:
factor = 7
reps = 5
min_hyperparam = 0.1
max_hyperparam = 10

In [4]:
import optuna

def evaluate(c):
    torch.cuda.empty_cache()
    gc.collect()

    consts.CONST_C = c

    avg_acc_test = 0.0
    for _ in range(reps):
        train_acc,test_acc, test_preds,test_gt = whole_pipeline_all(X,y, factor, factor, is_normalize_each_band=True, method_label_patch='most_common')
        avg_acc_test += test_acc/reps

    score = avg_acc_test
    return score

# Define the objective function for Optuna
def objective(trial):
    # Suggest a value for the hyperparameter within a given range
    hyperparameter = trial.suggest_float('c', min_hyperparam, max_hyperparam)
    
    # Evaluate the hyperparameter
    score = evaluate(hyperparameter)
    
    return score


In [12]:
# Create a study object and specify the direction of optimization
study = optuna.create_study(direction='maximize')  # or 'maximize' depending on your evaluation function

# Start the optimization
study.optimize(objective, n_trials=10)

# Print the best hyperparameter and its corresponding score
print("Best Hyperparameter: ", study.best_params['c'])
print("Best Score: ", study.best_value)


[I 2024-06-19 21:20:45,968] A new study created in memory with name: no-name-8eaf5ff0-4953-410f-a690-ef5f985c5f6b


XXXXXXX IN METHOD XXXXXXXXX


[W 2024-06-19 21:21:00,435] Trial 0 failed with parameters: {'c': 22.212299535386034} because of the following error: TypeError('zeros() received an invalid combination of arguments - got (tuple, device=torch.device, dtype=torch.dtype), but expected one of:\n * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)\n').
Traceback (most recent call last):
  File "/home/amirbourvine/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_442427/548659408.py", line 23, in objective
    score = evaluate(hyperparameter)
  File "/tmp/ipykernel_442427/548659408.py", line 11, in evaluate
    train_acc,test_acc, test_preds,test_gt = whole_pipeline_all(X,y, factor, factor, is_normalize

TypeError: zeros() received an invalid combination of arguments - got (tuple, device=torch.device, dtype=torch.dtype), but expected one of:
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
